In [8]:
!pip install numpy
!pip install automata-lib

You should consider upgrading via the 'c:\users\behrad\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\behrad\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [9]:
import math
import numpy as np

In [10]:
S = ["110","","","","0", "", "00","00","","","","10110","","","100"]

def merge(z, o):
    pz = 0
    po = 0
    l = []
    if o == []:
        o = [""];
    if z == []:
        z = [""];
    while pz < len(z) and po < len(o):
#         print(pz, z, po, o)
        if len(z[pz]) <= len(o[po]):
                    
                    l.append(z[pz])
                    pz+= 1
                    if(pz >= len(z)):
                        l.extend(o[po:])
                        break

        elif len(z[pz]) > len(o[po]):
            l.append(o[po])
            po+= 1
            if(po >= len(o)):
                l.extend(z[pz:])
                break
    if "" in l: l.remove("")
    return l

def build_list(s, u):
    longest_string = max(s, key=len)
    longest_string_len = (len(longest_string))
    if len(u) > longest_string_len:
        if [k for k in s if (u) in k]:
            return [u]
    zero = "0"
    one = "1"
    l = []
    bz = []
    bo = []
#     print(s, u)
    if any(st.startswith(u) for st in s):
        l.append(u)
    
    if any(st.startswith(u+zero) for st in s):
#         l.append(u + zero)
        bz = build_list(s, u + zero)
    
    if any(st.startswith(u+one) for st in s):
#         l.append(u + one)
        bo = build_list(s, u + one)
#         print(bo)
    res = l + merge(bz, bo)
    if "" in res: res.remove("")
    return res
build_list(S,"")

['0', '1', '00', '10', '11', '100', '101', '110', '1011', '10110']

In [11]:
def lambdaCalc(alpha):
    return math.sqrt((1/2)*math.log(2/alpha))

In [12]:
def fill(s, i):
    if len(s) < 1:
        return [""]*i
    return list(s) + [""]*(i - len(s))

def order(s, S):
    bl = build_list(S,"")
    return bl.index(s) + 2
#     return pow(2,len(s)) - 1 + int(s,2)
def is_leaf(S):
    if len(S) > 1:
        if isinstance(S[1], list):
            return False
    if len(S[0]) == 1:
        return True
    if isinstance(S[0][1],list):
        return False
    return True

def is_leaf2(S):
    if len(S) > 1:
        return False
    if len(S[0]) == 2:
        return True
    if isinstance(S[0][1],list):
        return False
    return True

def finished_here(s, d):
#     print([s if len(s) == d for string in s])
    result = list(filter(lambda string: len(string) == d+1, s))
    return result

def get_idx(n):
    return n[2]

In [13]:
transition_layer_num = 2#string - probability

def binaryBuildPTA(S, d, ch):
#     if len(set(S)) == 1:
#         result = []
#         result.append(fill(S, len(S)))
#         return [result, ch];
    longest_string = max(S, key=len)
    longest_string_len = (len(longest_string))
    if d >= longest_string_len:
        result = []
        result.append(S)
#         print("result: ", result, list(S))
        return result;
    matrix = np.empty([len(S), longest_string_len], dtype = str)
    for idx,s in enumerate(S):
        matrix[idx,:] = fill(s, longest_string_len)
    char_set = set(matrix[:,d])
    if d >= 0:
        if '' in char_set: char_set.remove('')

    list_subtree = [];
    end_nodes = len(S)
#     print("matrix: ", matrix)
    for k, c in enumerate(char_set):
        include_c_string = [S[j] for j,st in enumerate(matrix[:,d]) if matrix[j,d]==c]
#         print(include_c_string, d, np.sum([len(s) == d+1 for s in include_c_string]))
        
        end_nodes -= np.sum([len(s) == d+1 for s in include_c_string])
#         list_subtree.append([c, binaryBuildPTA(include_c_string,d+1, str(ch + c)), len(include_c_string), order(ch+c, S)])
        bpta = binaryBuildPTA(include_c_string,d+1, str(ch + c))
        
        list_subtree.append([str(ch + c), bpta, order(ch+c, all_strings), len(include_c_string), len(finished_here(include_c_string, d))])

    if d ==0:
        list_subtree = ["", list_subtree, 1, len(S), len(finished_here(S, d - 1))]
#     print(list_subtree)
    return list_subtree;

In [14]:
s = ["110","","","","0", "", "00","00","","","","10110","","","100"]
# s = np.array(["00","01","10","11"])
# s = np.array(["0","1",""])
all_strings = s
alpha = 0.8
lambda_ = lambdaCalc(alpha);
lambda_
binaryBuildPTA(s, 0,"")

['',
 [['0', [['00', [['00', '00']], 4, 2, 2]], 2, 3, 1],
  ['1',
   [['10',
     [['100', [['100']], 7, 1, 1],
      ['101',
       [['1011', [['10110', [['10110']], 11, 1, 1]], 10, 1, 0]],
       8,
       1,
       0]],
     5,
     2,
     0],
    ['11', [['110', [['110']], 9, 1, 1]], 6, 1, 0]],
   3,
   3,
   0]],
 1,
 15,
 9]

In [15]:
dict_empty = dict()
def get_states(t):
    if is_leaf(t):
        return ["q"+t[0]]
    l = ["q"+t[0]]
    for tr in t[1]:
        l = l + get_states(tr)
    return list(set(l))

bpta = binaryBuildPTA(S, 0,"")

def get_transitions(s, input_symbols, d):
    l = list(filter(lambda x: len(x) == d, s))
    if len(l) == 0:
        return dict_empty
    transitions = dict()
    for st in l:
        nexts = dict()
        for i in input_symbols:
            if st + i in s:
                nexts[i] = {st + i}
        transitions[st] = nexts
#     print(transitions)
    transitions = {**transitions, **get_transitions(s, input_symbols, d + 1)}
#     print(transitions)
#     print(transitions)
    transitions = {k: v for k, v in transitions.items() if v != dict_empty}
    return transitions

print(get_transitions(get_states(bpta), {"0", "1"}, 1))


print(get_states(bpta))

{'q': {'0': {'q0'}, '1': {'q1'}}, 'q0': {'0': {'q00'}}, 'q1': {'0': {'q10'}, '1': {'q11'}}, 'q11': {'0': {'q110'}}, 'q10': {'0': {'q100'}, '1': {'q101'}}, 'q101': {'1': {'q1011'}}, 'q1011': {'0': {'q10110'}}}
['q110', 'q0', 'q11', 'q10110', 'q', 'q10', 'q101', 'q1011', 'q1', 'q100', 'q00']


In [16]:
from automata.base.automaton import Automaton
from automata.fa.fa import FA
from automata.fa.nfa import NFA
# NFA which matches all binary strings ending in an odd number of '1's
nfa = NFA(
    states=set(get_states(bpta)),
    input_symbols={'0', '1'},
    transitions=get_transitions(get_states(bpta), {"0", "1"}, 1),
    initial_state="q",
    final_states=set(get_states(bpta))
)

In [17]:
dict_empty = dict()

def get_2(t):
#     print(t)
    if is_leaf(t):
        return dict_empty
    l = dict()
    l["q"+t[0]] = t[2]
    for tr in t[1]:
        l = {**l, **get_2(tr)}
    return (l)

def get_3(t):
#     print(t)
    if is_leaf(t):
        return dict_empty
    l = dict()
    l["q"+t[0]] = t[3]
    for tr in t[1]:
        l = {**l, **get_3(tr)}
    return (l)

def get_4(t):
#     print(t)
    if is_leaf(t):
        return dict_empty
    l = dict()
    l["q"+t[0]] = t[4]
    for tr in t[1]:
        l = {**l, **get_4(tr)}
    return (l)

get_4(bpta)

{'q': 9,
 'q0': 1,
 'q00': 2,
 'q1': 0,
 'q10': 0,
 'q100': 1,
 'q101': 0,
 'q1011': 0,
 'q10110': 1,
 'q11': 0,
 'q110': 1}

In [18]:
def different(n1, fn1, n2, fn2):    
    return abs(fn1/n1 - fn2/n2) > lambdaCalc(alpha)*(1/math.sqrt(n1) + 1/math.sqrt(n2))

In [20]:
states = get_states(bpta)
transitions = get_transitions(get_states(bpta), {"0", "1"}, 1)
get2 = (get_2(bpta))
get3 = (get_3(bpta))
get4 = (get_4(bpta))

print(states, transitions, get2, get3, get4)

def compatible(nfa, states, transitions, get2, get3, get4, n1, n2):
    if different(get3[n1], get4[n1], get3[n2], get4[n2]):
        return False
    print(transitions)
    char_list1 = transitions[n1].keys()#TODO ensemble this for different char lists
    char_list2 = transitions[n2].keys()
    for c in char_list1:
        e1 = e2 = False
        try:
            next_node1 = find_next(states, transitions, n1, c)
        except Exception:
            e1 = True
        try:
            next_node2 = find_next(states, transitions, n2, c)
        except Exception:
            e2 = True
        if (e1) != (e2):
            return False
        elif not (e1 or e2):
            if different(get3[next_node1], get4[next_node1], get3[next_node2], get4[next_node2]):
                return False

    return True

def end_at_node(S, n):
    return get_4(S)[n]

def comes_to_node(S, n):
    return get_3(S)[n]

def find_next(states, transitions, state, c):
    print("check")
    print(transitions)
    print(state, c)
    return list(transitions[state][c])[0]

S = ["110","","","","0", "", "00","00","","","","10110","","","100"]
#S = np.array(["00","01","10","11"])
all_strings = S
compatible(nfa, states, transitions, get2, get3, get4, states[0], states[1])

['q110', 'q0', 'q11', 'q10110', 'q', 'q10', 'q101', 'q1011', 'q1', 'q100', 'q00'] {'q': {'0': {'q0'}, '1': {'q1'}}, 'q0': {'0': {'q00'}}, 'q1': {'0': {'q10'}, '1': {'q11'}}, 'q11': {'0': {'q110'}}, 'q10': {'0': {'q100'}, '1': {'q101'}}, 'q101': {'1': {'q1011'}}, 'q1011': {'0': {'q10110'}}} {'q': 1, 'q0': 2, 'q00': 4, 'q1': 3, 'q10': 5, 'q100': 7, 'q101': 8, 'q1011': 10, 'q10110': 11, 'q11': 6, 'q110': 9} {'q': 15, 'q0': 3, 'q00': 2, 'q1': 3, 'q10': 2, 'q100': 1, 'q101': 1, 'q1011': 1, 'q10110': 1, 'q11': 1, 'q110': 1} {'q': 9, 'q0': 1, 'q00': 2, 'q1': 0, 'q10': 0, 'q100': 1, 'q101': 0, 'q1011': 0, 'q10110': 1, 'q11': 0, 'q110': 1}
{'q': {'0': {'q0'}, '1': {'q1'}}, 'q0': {'0': {'q00'}}, 'q1': {'0': {'q10'}, '1': {'q11'}}, 'q11': {'0': {'q110'}}, 'q10': {'0': {'q100'}, '1': {'q101'}}, 'q101': {'1': {'q1011'}}, 'q1011': {'0': {'q10110'}}}


KeyError: 'q110'

In [ ]:
def states_to_delete(transitions, s):
    if s not in transitions:
        return []
    transition_set = transitions[s]
    l = list(transition_set.values())
    l.append(s)
    for k in transition_set.values():
        l += states_to_delete(transitions, list(k)[0])
    return l

def delete_and_successors(transitions, s):
    to_delete_states = (states_to_delete(transitions, s))
    for state in to_delete_states:
        transitions.pop(state, None)
    return transitions
states = list(set(get_states(bpta)))
transitions = get_transitions(get_states(bpta), {"0", "1"}, 1)
s2 = "q0"
delete_and_successors(transitions, s2)

In [ ]:
states = list(set(get_states(bpta)))
get2 = (get_2(bpta))
get3 = (get_3(bpta))
get4 = (get_4(bpta))


s1 = "q"
s2 = "q0"

if compatible(nfa, states, transitions, get2, get3, get4, s1, s2):
    states.remove(s2)
    transitions = delete_and_successors(transitions, s2)

    for key in transitions.keys():
        for k in transitions[key].keys():
            if list(transitions[key][k])[0] == s2:         
                transitions[key][k] = s1

print(states)
print(transitions)